In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Research Project

/content/drive/My Drive/Research Project


In [ ]:
pip install --upgrade transformers

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (3.5.1)


In [ ]:
pip install simpletransformers

In [ ]:
# pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pip install -U mittens

Requirement already up-to-date: mittens in /usr/local/lib/python3.6/dist-packages (0.2)


In [ ]:
from numpy import asarray
from numpy import zeros

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold

from statistics import mean, stdev, median, mode

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

from mittens import GloVe, Mittens
from hyperopt import fmin, tpe, hp

In [ ]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import MaxPool3D, AveragePooling3D

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [ ]:
from hyperopt import fmin, tpe, hp

In [ ]:
# pip install autocorrect

In [ ]:
# nltk.download('punkt')

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout, MaxPool1D
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Bidirectional
import tensorflow.keras.backend as K

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

### Getting Data

In [ ]:
# data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [ ]:
data_path = r'/content/drive/My Drive/Research Project/Data'

In [ ]:
train_pr_df = pd.read_csv(os.path.join(data_path, 'Obama Training Data.csv'), usecols = [1,2], names= ['text', 'labels'], header  = 0).dropna()
train_pr_df['labels'] = train_pr_df['labels'].apply(lambda x: 0 if x == 'Positive' else (2 if x == 'Negative' else 1))
val_pr_df = pd.read_csv(os.path.join(data_path, 'Obama Validation Data.csv'), usecols = [1,2], names= ['text', 'labels'], header  = 0).dropna()
val_pr_df['labels'] = val_pr_df['labels'].apply(lambda x: 0 if x == 'Positive' else (2 if x == 'Negative' else 1))

In [ ]:
train_pr_df.head()

,text,labels
0,obama need b remove dangerous white house czar...,2
1,sidenote obama profile sexy back debate,0
2,hillary take bullet obama n buck still fall,2
3,morgan freeman obama best commercial ever need...,0
4,bitch like obama bitch want food stamp lmao,1


In [ ]:
val_pr_df.head()

,text,labels
0,hell obama can not even take care one border p...,2
1,obama save auto industry enough say team obama,0
2,great obama hope take today sport section cheat,2
3,appreciate obama capitalisizes fact black,2
4,ann romney michelle obama dress second debate ...,1


In [ ]:
train1_pr_df = pd.read_csv(os.path.join(data_path, 'Obama Training1 Data.csv'), usecols = [1,2],names= ['text', 'labels'], header  = 0).dropna()
train1_pr_df['labels'] = train1_pr_df['labels'].apply(lambda x: 0 if x == 'Positive' else (2 if x == 'Negative' else 1))

In [ ]:
def evaluation(labels, preds):

  f1_temp = np.round(f1_score(labels, preds, average = None),3)

  accuracy = round(accuracy_score(labels, preds),3) 


  return np.round(f1_temp,3), round(accuracy,3)

In [ ]:
train1_pr_df.head()

,text,labels
0,kirkpatrick wear baseball cap embroider obama ...,1
1,obama debate cracker as cracker tonight tune t...,0
2,miss point afraid understand big picture dont ...,1
3,raise democrat leave party year ago lifetime n...,2
4,obama camp can not afford low expectation toni...,1



## Building Transformer Models

In [ ]:
# max_length = 61
# batch_size = 16

# par_dict = {'max_length': max_length, 'batch_size': batch_size}
# print(par_dict)
# model_args = ClassificationArgs(num_train_epochs=2, train_batch_size = batch_size, overwrite_output_dir=True, max_seq_length = max_length)
# model = ClassificationModel(model_type='roberta', model_name='roberta-base', use_cuda=True, num_labels=3, args=model_args)
# model.train_model(train_pr_df)

# y_pred = model.predict(val_pr_df['text'])[0]


# f1_temp = np.round(f1_score(val_pr_df['labels'], y_pred, average = None),3)
# recall = np.round(recall_score(val_pr_df['labels'], y_pred, average = None),3)
# precision = np.round(precision_score(val_pr_df['labels'], y_pred, average = None),3)

# f1_Positive = f1_temp[0]
# f1_Neutral = f1_temp[1]
# f1_Negative = f1_temp[2]

# accuracy = round(accuracy_score(val_pr_df['labels'], y_pred),3) 
# eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)   
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print('\n')
# print(eval_dict)
# print("f1" + str(f1_temp))
# print('recall' + str(recall))
# print('Precision' + str(precision))

### Cross Validation Function for Transformers


In [ ]:
def cross_valid_TRA(X, train_batch_size,max_seq_length):
  f1_Positive  =[]
  f1_Neutral =[]
  f1_Negative =[]
  acc =[]
  eval =[]
  cv = KFold(n_splits=5,shuffle=True)
  for train_index, val_index in cv.split(X):
  #     print("Train Index: ", train_index, "\n")
  #     print("Test Index: ", test_index)
    # print(X.iloc[val_index]['text'])
  #     print(f)

    X_train1, X_val = X.iloc[train_index], X.iloc[val_index]
    model_args = ClassificationArgs(num_train_epochs=2, train_batch_size = train_batch_size, overwrite_output_dir=True, max_seq_length = max_seq_length)
    model = ClassificationModel(model_type='roberta', model_name='cardiffnlp/twitter-roberta-base-sentiment', use_cuda=True, num_labels=3, args=model_args)
    model.train_model(X_train1)

    result, model_outputs, wrong_predictions = model.eval_model(X_val, eval_score=evaluation)
    f1_temp = result['eval_score'][0]
    accuracy = result['eval_score'][1]
    eval_score = round(mean([0.4*accuracy, 1.6*f1_temp[0]]),3)
    f1_Positive.append(f1_temp[0])
    f1_Neutral.append(f1_temp[1])
    f1_Negative.append(f1_temp[2])

    acc.append(accuracy)
    eval.append(eval_score)

    # print(f1_temp)
    # print(accuracy)
    # print(fas)
  
  return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3), round(mean(eval),3)


In [ ]:
def objective_func_CNN(args):
    max_seq_length = args['max_seq_length']
    train_batch_size = args['train_batch_size']
    # learning_rate = args['learning_rate']
    # epochs = args['epochs']

    par_dict = {'max_length': max_seq_length, 'batch_size': train_batch_size}
    print(par_dict)

    accuracy, f1_Positive, f1_Neutral, f1_Negative, eval_score = cross_valid_TRA(train1_pr_df, train_batch_size,max_seq_length)   
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print('\n')
    print(eval_dict)


    return -(eval_score)

space = {'max_seq_length': hp.choice('max_seq_length',range(5,65,5)),  
        'train_batch_size': hp.choice('train_batch_size', [16, 32, 64])
        #  'epochs': hp.choice('epochs',range(1,3)) 
        #  'learning_rate': hp.uniform('learning_rate', 0,0.0005)
        }                  
                                                              
best_T = fmin(objective_func_CNN, space, algo=tpe.suggest, max_evals=15)
print(best_T)

{'max_length': 60, 'batch_size': 64}
  0%|          | 0/15 [00:00<?, ?it/s, best loss: ?]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



Exception ignored in: 
<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f54b8673358>>
Traceback (most recent call last):

  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__

self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers

w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join

assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError
: 
can only join a child process
Exception ignored in: 
<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f54bb157080>>
Traceback (most recent call last):

  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__

self._shu

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'accuracy': 0.632, 'f1_pos': 0.654, 'f1_neu': 0.564, 'f1_neg': 0.67, 'eval_score': 0.65}
100%|██████████| 15/15 [1:59:47<00:00, 479.18s/it, best loss: -0.655]
{'max_seq_length': 10, 'train_batch_size': 0}


In [ ]:
{'max_length': 55, 'batch_size': 16}
{'accuracy': 0.636, 'f1_pos': 0.66, 'f1_neu': 0.575, 'f1_neg': 0.669, 'eval_score': 0.655}

In [ ]:
 'cardif nlp'
{'max_length': 35, 'batch_size': 16}
{'accuracy': 0.638, 'f1_pos': 0.654, 'f1_neu': 0.577, 'f1_neg': 0.68, 'eval_score': 0.651}

{'accuracy': 0.638,
 'eval_score': 0.651,
 'f1_neg': 0.68,
 'f1_neu': 0.577,
 'f1_pos': 0.654}

In [ ]:
{'max_length': 54, 'batch_size': 128}
{'accuracy': 0.6, 'f1_pos': 0.641, 'f1_neu': 0.526, 'f1_neg': 0.629, 'eval_score': 0.62}


{'accuracy': 0.6,
 'eval_score': 0.62,
 'f1_neg': 0.629,
 'f1_neu': 0.526,
 'f1_pos': 0.641}

In [ ]:
{'max_length': 61, 'batch_size': 16}
{'accuracy': 0.602, 'f1_pos': 0.637, 'f1_neu': 0.518, 'f1_neg': 0.652, 'eval_score': 0.619}

{'accuracy': 0.602,
 'eval_score': 0.619,
 'f1_neg': 0.652,
 'f1_neu': 0.518,
 'f1_pos': 0.637}

In [ ]:
print(adfadf)

NameError: ignored

In [ ]:
model.train_model(train_pr_df)

In [ ]:
def evaluation(labels, preds):

  f1_temp = np.round(f1_score(labels, preds, average=None),3)

  f1_Positive = f1_temp[0]
  f1_Neutral = f1_temp[1]
  f1_Negative = f1_temp[2]
  accuracy = round(accuracy_score(labels, preds),3) 
  eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3) 
  return eval_score

In [ ]:
scores2, model_outputs, wrong_predictions = model.eval_model(val_pr_df, f1=evaluation, acc=accuracy_score, verbose  = False)

In [ ]:
scores2

In [ ]:
# y_pred  =  model.predict(val_pr_df['text'])[0]

In [ ]:
# import logging
# from statistics import mean, mode

# import pandas as pd
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.model_selection import train_test_split

# import wandb
# from simpletransformers.classification import ClassificationArgs, ClassificationModel
# # from utils import load_rte_data_file

# sweep_config = {
#     "name": "vanilla-sweep-batch-16",
#     "method": "bayes",
#     "metric": {"name": "accuracy", "goal": "maximize"},
#     "parameters": {
#         "num_train_epochs": {"min": 1, "max": 20},
#         "learning_rate": {"min": 0, "max": 4e-4},
#     },
#     "early_terminate": {"type": "hyperband", "min_iter": 6,},
# }

# sweep_id = wandb.sweep(sweep_config, project="RTE - Hyperparameter Optimization")

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Preparing train data
# train_df = train_pr_df
# eval_df = val_pr_df

# model_args = ClassificationArgs()
# model_args.eval_batch_size = 8
# model_args.evaluate_during_training = True
# model_args.evaluate_during_training_silent = False
# model_args.evaluate_during_training_steps = 2
# model_args.learning_rate = 4e-4
# model_args.manual_seed = 4
# model_args.max_seq_length = 256
# model_args.multiprocessing_chunksize = 5000
# model_args.no_cache = True
# model_args.no_save = True
# model_args.num_train_epochs = 10
# model_args.overwrite_output_dir = True
# model_args.reprocess_input_data = True
# model_args.train_batch_size = 16
# model_args.gradient_accumulation_steps = 2
# model_args.train_custom_parameters_only = False
# model_args.labels_list = ["not_entailment", "entailment"]
# model_args.wandb_project = "RTE - Hyperparameter Optimization"


# def train():
#     # Initialize a new wandb run
#     wandb.init()

#     # Create a TransformerModel
#     model = ClassificationModel(
#         "roberta",
#         "roberta-large",
#         use_cuda=True,
#         args=model_args,
#         sweep_config=wandb.config,
#     )

#     # Train the model
#     model.train_model(
#         train_df,
#         eval_df=eval_df,
#         accuracy=lambda truth, predictions: accuracy_score(
#             truth, [round(p) for p in predictions]
#         ),
#     )

#     # Sync wandb
#     wandb.join()

# wandb.agent(sweep_id, train)

In [ ]:
# import logging
# from statistics import mean, mode

# import pandas as pd
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.model_selection import train_test_split

# import wandb
# from simpletransformers.classification import ClassificationArgs, ClassificationModel
# # from utils import load_rte_data_file

# sweep_config = {
#     "name": "vanilla-sweep-batch-16",
#     "method": "bayes",
#     "metric": {"name": "accuracy", "goal": "maximize"},
#     "parameters": {
#         "num_train_epochs": {"min": 1, "max": 40},
#         "learning_rate": {"min": 0, "max": 4e-4},
#     },
#     "early_terminate": {"type": "hyperband", "min_iter": 6,},
# }

# sweep_id = wandb.sweep(sweep_config, project="RTE - Hyperparameter Optimization")

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Preparing train data
# train_df = train_pr_df
# eval_df = val_pr_df

# model_args = ClassificationArgs()
# model_args.eval_batch_size = 8
# model_args.evaluate_during_training = True
# model_args.evaluate_during_training_silent = False
# model_args.evaluate_during_training_steps = 1000
# model_args.learning_rate = 4e-4
# model_args.manual_seed = 4
# model_args.max_seq_length = 256
# model_args.multiprocessing_chunksize = 5000
# model_args.no_cache = True
# model_args.no_save = True
# model_args.num_train_epochs = 10
# model_args.overwrite_output_dir = True
# model_args.reprocess_input_data = True
# model_args.train_batch_size = 16
# model_args.gradient_accumulation_steps = 2
# model_args.train_custom_parameters_only = False
# model_args.labels_list = ["not_entailment", "entailment"]
# model_args.wandb_project = "RTE - Hyperparameter Optimization"


# def train():
#     # Initialize a new wandb run
#     wandb.init()

#     # Create a TransformerModel
#     model = ClassificationModel(
#         "roberta",
#         "roberta-large",
#         use_cuda=True,
#         args=model_args,
#         sweep_config=wandb.config,
#     )

#     # Train the model
#     model.train_model(
#         train_df,
#         eval_df=eval_df,
#         accuracy=lambda truth, predictions: accuracy_score(
#             truth, [round(p) for p in predictions]
#         ),
#     )

#     # Sync wandb
#     wandb.join()

# wandb.agent(sweep_id, train)